# Proyecto 1
## Cluster
### Grupo 2

##### Sección de librerías.

In [8]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans, AgglomerativeClustering
from sklearn.preprocessing import OneHotEncoder, StandardScaler

##### Importación de datos.

In [3]:
df = pd.read_csv('DATOS_PROYECTO_1/DB_LIMPIA.csv', sep = ";", index_col = 0)
df.columns

Index(['VIC_SEXO', 'VIC_EDAD', 'TOTAL_HIJOS', 'VIC_ESCOLARIDAD', 'VIC_EST_CIV',
       'VIC_GRUPET', 'VIC_TRABAJA', 'VIC_DISC', 'VIC_REL_AGR',
       'OTRAS_VICTIMAS', 'HEC_DIA', 'HEC_MES', 'HEC_ANO', 'HEC_AREA',
       'HEC_TIPAGRE', 'INST_DONDE_DENUNCIO', 'AGR_SEXO', 'AGR_EDAD',
       'AGR_ESCOLARIDAD', 'AGR_EST_CIV', 'AGR_GRUPET', 'AGR_TRABAJA',
       'INST_DENUN_HECHO', 'MEDIDAS_SEGURIDAD'],
      dtype='object')

##### Situación problemática
En el conjunto tenemos información acerca del hecho, de las víctimas y agresores (según la denuncia), de denuncias de casos de violencia intrafamiliar que se han realizado entre los años 2009 a 2019. Para este conjunto de datos se buscará analizar si hay algunas características, de las personas, en las cuales podemos agrupar y diferenciar si es víctima o agresor. Este análisis no se hace con el fin de generar prejuicios, ni de perjudicar a ninguna persona cuyas características puedan ser similares a las de un grupo, solo buscamos observar si es posible realizar esta agrupación.

##### Problema científico.
¿Es posible agrupar a las personas en víctimas y en agresores según las características mostradas en el conjunto de datos?

##### Objetivo general.
Agrupar a individuos en la situación de víctima y agresor según las características de los individuos, mediante el uso de 2 algoritmos de agrupación, los cuales se verificarán con las etiquetas reales, con el fin de verificar si existen características compartidas entre víctimas y entre agresores.

##### Objetivos específicos.
- Agrupar a individuos en la situación de víctima y agresor según las características de los individuos, haciendo uso del algoritmos de KMeans, con el fin de verificar si existen características compartidas entre víctimas y entre agresores.
- Agrupar a individuos en la situación de víctima y agresor según las características de los individuos, haciendo uso del algoritmos de AgglomerativeClustering, con el fin de verificar si existen características compartidas entre víctimas y entre agresores.


In [6]:
df_victimas = df.loc[:, ['VIC_SEXO', 'VIC_EDAD', 'VIC_ESCOLARIDAD', 'VIC_EST_CIV', 'VIC_GRUPET', 'VIC_TRABAJA']]
df_victimas.columns = ['SEXO', 'EDAD', 'ESCOLARIDAD', 'ESTADO_CIVIL', 'GRUPO_ETNICO', 'TRABAJA']
df_victimas['ETIQUETA'] = 'VICTIMA'

df_agresores = df.loc[:, ['AGR_SEXO', 'AGR_EDAD', 'AGR_ESCOLARIDAD', 'AGR_EST_CIV', 'AGR_GRUPET', 'AGR_TRABAJA']]
df_agresores.columns = ['SEXO', 'EDAD', 'ESCOLARIDAD', 'ESTADO_CIVIL', 'GRUPO_ETNICO', 'TRABAJA']
df_agresores['ETIQUETA'] = 'AGRESOR'

df_cluster = pd.concat([df_victimas, df_agresores])
df_cluster.head(5)

,SEXO,EDAD,ESCOLARIDAD,ESTADO_CIVIL,GRUPO_ETNICO,TRABAJA,ETIQUETA
0,Mujeres,11,Primaria,Desconocido,Ladino,No,VICTIMA
1,Hombres,4,Desconocido,Desconocido,Ladino,Desconocido,VICTIMA
2,Hombres,11,Primaria,Desconocido,Ladino,No,VICTIMA
3,Mujeres,6,Desconocido,Desconocido,Ladino,Desconocido,VICTIMA
4,Hombres,11,Primaria,Desconocido,Ladino,No,VICTIMA


In [14]:
cat_cols = ["SEXO", "ESCOLARIDAD", "ESTADO_CIVIL", "GRUPO_ETNICO", "TRABAJA"]
num_cols = ["EDAD"]

ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
X_cat = ohe.fit_transform(df_cluster[cat_cols])

# 3) Escalar EDAD (importante en clustering)
scaler = StandardScaler()
X_num = scaler.fit_transform(df_cluster[num_cols])

# 4) Unir numéricas + one-hot
X = np.hstack([X_num, X_cat])

feature_names = np.concatenate([num_cols, ohe.get_feature_names_out(cat_cols)])
X_df = pd.DataFrame(X, columns=feature_names, index=df_cluster.index)
X_df.head()

,EDAD,SEXO_Hombres,SEXO_Mujeres,ESCOLARIDAD_Básico,ESCOLARIDAD_Desconocido,ESCOLARIDAD_Diversificado,ESCOLARIDAD_Ninguna,ESCOLARIDAD_Primaria,ESCOLARIDAD_Universidad,ESTADO_CIVIL_Casado,...,ESTADO_CIVIL_Viudo,GRUPO_ETNICO_Desconocido,GRUPO_ETNICO_Garifuna,GRUPO_ETNICO_Ladino,GRUPO_ETNICO_Maya,GRUPO_ETNICO_Otro,GRUPO_ETNICO_Xinca,TRABAJA_Desconocido,TRABAJA_No,TRABAJA_Si
0,-1.585821,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-2.112327,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
2,-1.585821,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,-1.961897,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-1.585821,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
